In [41]:
# words = ['Hello','fine','mother','father']
words = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'again', 'age', 'all done', 'b', 'bathroom', 'brother', 'busy', 'c', 'call','cat', 'change', 'coat', 'cold', 'cow', 'cry', 'cup', 'd', 'danger', 'deaf','Dog', 'Do not, does not', 'Nice to meet you', 'Forget', 'drink', 'e', 'eat', 'family', 'father', 'f', 'favorite', 'fine',  'food', 'friend', 'g', 'go', 'goodbye', 'great', 'h','happy', 'have', 'havent', 'hearing', 'hello', 'How are you', 'hungary','i', 'i or me', 'intelligent', 'j', 'k', 'l', 'late','learn', 'like', 'line', 'm','meet', 'more', 'mother', 'n', 'name', 'no', 'nothing', 'need', 'o', 'p', 'please', 'q', 'quiet', 'r', 'ready', 'restaurant', 's', 'school', 'see you later', 'sign', 'sister', 'sleep','see', 'slow', 'so-so', 'sorry',  'sad', 'scare', 'you', 't', 'takecare', 'teacher', 'thankyou', 'time', 'u', 'understand', 'v', 'w', 'water', 'what', 'where', 'work', 'wrong', 'x', 'y', 'yes', 'z']

# **Zoom in and out with w and e**

In [2]:
import cv2

# Initialize camera
cap = cv2.VideoCapture(0)

# Set zoom scale
zoom_scale = 1.0
zoom_step = 0.1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the height and width of the frame
    height, width = frame.shape[:2]

    # Calculate the cropping dimensions
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)

    # Calculate the center point for cropping
    x_center = width // 2
    y_center = height // 2

    # Crop the frame to simulate zooming in
    cropped_frame = frame[y_center - new_height//2: y_center + new_height//2,
                          x_center - new_width//2: x_center + new_width//2]

    # Resize the cropped frame back to the original size
    resized_frame = cv2.resize(cropped_frame, (width, height))

    # Show the resulting frame
    cv2.imshow('Camera Zoom', resized_frame)

    # Get user input to zoom in and out
    key = cv2.waitKey(1)
    if key == ord('w') and zoom_scale < 2.0:  # Zoom in
        zoom_scale += zoom_step
    elif key == ord('e') and zoom_scale > 1.0:  # Zoom out
        zoom_scale -= zoom_step
    elif key == ord('q'):  # Quit the loop
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


# **Check the distance between person and camera**

## Calculate distance from multiple persons

In [3]:
import cv2

# Known distance from camera to object (in cm) and known face width (in cm)
KNOWN_DISTANCE = 60.0  # Example: 60 cm from the camera
KNOWN_WIDTH = 16.0  # Example: average face width is 16 cm

# Load the face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to estimate focal length
def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

# Function to estimate distance from camera
def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

# Initialize camera
cap = cv2.VideoCapture(0)

# Pre-calculate the focal length with a sample face width (adjust accordingly)
sample_face_width = 200  # Example: width of a face in pixels at KNOWN_DISTANCE
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Estimate the distance to the face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Show the frame
    cv2.imshow('Distance Estimation', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


## Calculate distance from single person

In [6]:
import cv2

# Known distance from camera to object (in cm) and known face width (in cm)
KNOWN_DISTANCE = 60.0  # Example: 60 cm from the camera
KNOWN_WIDTH = 16.0  # Example: average face width is 16 cm

# Load the face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to estimate focal length
def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

# Function to estimate distance from camera
def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

# Initialize camera
cap = cv2.VideoCapture(0)

# Pre-calculate the focal length with a sample face width (adjust accordingly)
sample_face_width = 200  # Example: width of a face in pixels at KNOWN_DISTANCE
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        # Select the face with the largest width (closest to the camera)
        largest_face = max(faces, key=lambda face: face[2])  # face[2] is the width

        (x, y, w, h) = largest_face

        # Draw rectangle around the largest face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Estimate the distance to the largest face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Display the distance on the frame
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Show the frame
    cv2.imshow('Distance Estimation', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


## Auto zoom in and out from 70cm to 100cm

In [29]:
import cv2

# Known distance from camera to object (in cm) and known face width (in cm)
KNOWN_DISTANCE = 60.0  # Example: 60 cm from the camera
KNOWN_WIDTH = 16.0  # Example: average face width is 16 cm
TARGET_MIN_DISTANCE = 80.0  # Minimum target distance
TARGET_MAX_DISTANCE = 90.0  # Maximum target distance
ZOOM_STEP_IN = 0.1  # Step for zooming in
ZOOM_STEP_OUT = 0.1  # Step for zooming out

# Load the face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to estimate focal length
def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

# Function to estimate distance from camera
def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

# Initialize camera
cap = cv2.VideoCapture(0)

# Pre-calculate the focal length with a sample face width (adjust accordingly)
sample_face_width = 200  # Example: width of a face in pixels at KNOWN_DISTANCE
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

# Set initial zoom scale
zoom_scale = 1.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the height and width of the frame
    height, width = frame.shape[:2]

    # Apply zooming effect (cropping and resizing the frame)
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)

    x_center = width // 2
    y_center = height // 2

    cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                          x_center - new_width // 2:x_center + new_width // 2]
    resized_frame = cv2.resize(cropped_frame, (width, height))

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        # Select the face with the largest width (closest to the camera)
        largest_face = max(faces, key=lambda face: face[2])  # face[2] is the width

        (x, y, w, h) = largest_face

        # Draw rectangle around the largest face
        cv2.rectangle(resized_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Estimate the distance to the largest face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Display the distance on the frame
        cv2.putText(resized_frame, f"Distance: {distance:.2f} cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Zoom in if distance > 90 cm to bring it to the range 80-90 cm
        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:  # Zoom in
            zoom_scale += ZOOM_STEP_IN
            print(f"Zooming in to adjust distance, current zoom scale: {zoom_scale}")

        # Zoom out if distance < 80 cm to bring it to the range 80-90 cm
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:  # Zoom out
            zoom_scale -= ZOOM_STEP_OUT
            print(f"Zooming out to adjust distance, current zoom scale: {zoom_scale}")

        # No zooming if the distance is within the 80-90 cm range
        elif TARGET_MIN_DISTANCE <= distance <= TARGET_MAX_DISTANCE:
            print("Distance is within acceptable range (80-90 cm), no zooming required.")

    # Show the frame
    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


In [27]:
import cv2

KNOWN_DISTANCE = 60.0
KNOWN_WIDTH = 16.0
TARGET_MIN_DISTANCE = 80.0
TARGET_MAX_DISTANCE = 90.0
ZOOM_STEP_IN = 0.1
ZOOM_STEP_OUT = 0.1

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

cap = cv2.VideoCapture(0)
sample_face_width = 200
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)
zoom_scale = 1.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)
    x_center = width // 2
    y_center = height // 2

    cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                          x_center - new_width // 2:x_center + new_width // 2]
    resized_frame = cv2.resize(cropped_frame, (width, height))

    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        largest_face = max(faces, key=lambda face: face[2])
        (x, y, w, h) = largest_face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:
            zoom_scale += ZOOM_STEP_IN
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:
            zoom_scale -= ZOOM_STEP_OUT

    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2

KNOWN_DISTANCE = 60.0
KNOWN_WIDTH = 16.0
TARGET_MIN_DISTANCE = 80.0
TARGET_MAX_DISTANCE = 90.0
ZOOM_STEP_IN = 0.1
ZOOM_STEP_OUT = 0.1

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

cap = cv2.VideoCapture(0)
sample_face_width = 200
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)
zoom_scale = 1.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        # Select the largest face
        largest_face = max(faces, key=lambda face: face[2])
        (x, y, w, h) = largest_face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Adjust zoom based on distance
        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:
            zoom_scale += ZOOM_STEP_IN
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:
            zoom_scale -= ZOOM_STEP_OUT

        # Crop the frame so the top of the face is at the top of the frame
        y_top = max(0, y - int(0.1 * h))  # Small margin above the head
        y_bottom = min(height, y + h + (height - (y + h)))  # Extend downwards to include body

        cropped_frame = frame[y_top:y_bottom, :]
        resized_frame = cv2.resize(cropped_frame, (width, height))
    else:
        # If no face detected, revert to the default cropped frame
        x_center = width // 2
        y_center = height // 2
        cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                              x_center - new_width // 2:x_center + new_width // 2]
        resized_frame = cv2.resize(cropped_frame, (width, height))

    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2

KNOWN_DISTANCE = 60.0
KNOWN_WIDTH = 16.0
TARGET_MIN_DISTANCE = 80.0
TARGET_MAX_DISTANCE = 90.0
ZOOM_STEP_IN = 0.1
ZOOM_STEP_OUT = 0.1
MARGIN_ABOVE_HEAD = 70  # Adjust this value for more or less space above the head

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

cap = cv2.VideoCapture(0)
sample_face_width = 200
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)
zoom_scale = 1.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        # Select the largest face
        largest_face = max(faces, key=lambda face: face[2])
        (x, y, w, h) = largest_face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Adjust zoom based on distance
        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:
            zoom_scale += ZOOM_STEP_IN
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:
            zoom_scale -= ZOOM_STEP_OUT

        # Adjust the frame: Include margin above the head
        y_top = max(0, y - MARGIN_ABOVE_HEAD)  # Add a margin above the head
        y_bottom = min(height, y + h + (height - (y + h)))  # Extend downwards to include body

        cropped_frame = frame[y_top:y_bottom, :]
        resized_frame = cv2.resize(cropped_frame, (width, height))
    else:
        # If no face is detected, revert to default cropping
        x_center = width // 2
        y_center = height // 2
        cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                              x_center - new_width // 2:x_center + new_width // 2]
        resized_frame = cv2.resize(cropped_frame, (width, height))

    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# **Place Head on Top**

In [15]:
import cv2

# Margin above the head (in pixels)
MARGIN_ABOVE_HEAD = 30  

# Load the Haar Cascade for initial face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

# Initialize face tracker (KCF is one of the options available in OpenCV)
tracker = cv2.TrackerKCF_create()
tracking = False  # To indicate if we are tracking a face

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if not tracking:  # Detect face if we are not currently tracking
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        if len(faces) > 0:
            # Select the largest face
            largest_face = max(faces, key=lambda face: face[2] * face[3])
            (x, y, w, h) = largest_face

            # Initialize the tracker with the detected face
            tracker = cv2.TrackerKCF_create()  # Reset the tracker
            tracker.init(frame, (x, y, w, h))
            tracking = True

    if tracking:  # If tracking is active, update the tracker
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in bbox]

            # Adjust the frame: Crop to position the head at the top with a small margin
            y_top = max(0, y - MARGIN_ABOVE_HEAD)
            y_bottom = min(frame.shape[0], y_top + h + (frame.shape[0] - (y + h)))
            cropped_frame = frame[y_top:y_bottom, :]

            # Resize the cropped frame to fit the original window size
            resized_frame = cv2.resize(cropped_frame, (frame.shape[1], frame.shape[0]))
        else:
            # If tracking fails, stop tracking and switch back to face detection
            tracking = False
            resized_frame = frame
    else:
        resized_frame = frame

    # Display the adjusted frame
    cv2.imshow('Head Adjusted to Top', resized_frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


# **Adjust head and zoom by message**

## **With Data**

In [4]:
import cv2
import numpy as np
import time

# Known distance from camera to object (in cm) and known face width (in cm)
KNOWN_DISTANCE = 60.0  # Example: 60 cm from the camera
KNOWN_WIDTH = 16.0  # Example: average face width is 16 cm
TARGET_MIN_DISTANCE = 80.0  # Minimum target distance
TARGET_MAX_DISTANCE = 90.0  # Maximum target distance
ZOOM_STEP_IN = 0.1  # Step for zooming in
ZOOM_STEP_OUT = 0.1  # Step for zooming out
HEAD_TOP_THRESHOLD_PIXELS = 100  # Margin above the head in pixels

# Load the face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to estimate focal length
def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

# Function to estimate distance from camera
def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

# Initialize camera
cap = cv2.VideoCapture(0)

# Pre-calculate the focal length with a sample face width (adjust accordingly)
sample_face_width = 200  # Example: width of a face in pixels at KNOWN_DISTANCE
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

# Set initial zoom scale
zoom_scale = 1.0

# Variables for animation
pulse_alpha = 0.4
pulse_direction = 1  # 1 for increasing transparency, -1 for decreasing
pulse_rate = 0.02  # How fast the transparency changes

arrow_pulse = 0  # Value to control arrow length for the pulse effect
arrow_pulse_direction = 1  # 1 for growing, -1 for shrinking
arrow_pulse_max = 40  # Maximum pulse size for the arrow
arrow_pulse_rate = 2  # Speed of the arrow animation

# Timer variables for blinking text
blink_start_time = time.time()
blink_duration = 1.0  # Blinking every 1 second

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get the height and width of the frame
    height, width = frame.shape[:2]

    # Apply zooming effect (cropping and resizing the frame)
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)

    x_center = width // 2
    y_center = height // 2

    cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                          x_center - new_width // 2:x_center + new_width // 2]
    resized_frame = cv2.resize(cropped_frame, (width, height))

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        # Select the face with the largest width (closest to the camera)
        largest_face = max(faces, key=lambda face: face[2])  # face[2] is the width
        (x, y, w, h) = largest_face

        # Draw rectangle around the largest face
        cv2.rectangle(resized_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Estimate the distance to the largest face
        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        # Display the distance on the frame
        cv2.putText(resized_frame, f"Distance: {distance:.2f} cm", (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Zoom in if distance > 90 cm to bring it to the range 80-90 cm
        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:  # Zoom in
            zoom_scale += ZOOM_STEP_IN
            print(f"Zooming in to adjust distance, current zoom scale: {zoom_scale}")

        # Zoom out if distance < 80 cm to bring it to the range 80-90 cm
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:  # Zoom out
            zoom_scale -= ZOOM_STEP_OUT
            print(f"Zooming out to adjust distance, current zoom scale: {zoom_scale}")

        # No zooming if the distance is within the 80-90 cm range
        elif TARGET_MIN_DISTANCE <= distance <= TARGET_MAX_DISTANCE:
            print("Distance is within acceptable range (80-90 cm), no zooming required.")

        # Calculate the distance from the top of the detected face to the top of the frame
        head_top_distance = y  # The y-coordinate of the top of the detected face
        
        # Calculate the effective threshold after applying zoom
        effective_threshold = HEAD_TOP_THRESHOLD_PIXELS * zoom_scale

        # Check if the distance exceeds the effective threshold (100 pixels)
        if head_top_distance > effective_threshold:
            # Adjust transparency pulse
            pulse_alpha += pulse_direction * pulse_rate
            if pulse_alpha >= 0.6 or pulse_alpha <= 0.1:  # Bounce effect for alpha
                pulse_direction *= -1  # Reverse direction

            # Draw a semi-transparent red overlay with pulsing effect
            overlay = resized_frame.copy()
            cv2.rectangle(overlay, (0, 0), (width, int(head_top_distance)), (0, 0, 255), -1)
            resized_frame = cv2.addWeighted(overlay, pulse_alpha, resized_frame, 1 - pulse_alpha, 0)

            # Adjust arrow pulsing effect (growing and shrinking)
            arrow_pulse += arrow_pulse_direction * arrow_pulse_rate
            if arrow_pulse >= arrow_pulse_max or arrow_pulse <= 0:
                arrow_pulse_direction *= -1

            # Draw an arrow pointing up with pulsing size
            arrow_start = (x + w // 2, y + h + 10)
            arrow_end = (x + w // 2, y + h + 10 + arrow_pulse)
            cv2.arrowedLine(resized_frame, arrow_end, arrow_start, (0, 0, 255), 4, tipLength=0.3)

            # Center the "Reduce upper distance" text in the red rectangle
            text = "Reduce upper distance"
            font_scale = 1
            font_thickness = 2
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
            text_x = (width - text_size[0]) // 2
            text_y = (int(head_top_distance) - text_size[1]) // 2 + text_size[1]

            # Blink the message every second
            current_time = time.time()
            if int(current_time - blink_start_time) % 2 == 0:  # Blink every second
                # Draw bold text (white)
                cv2.putText(resized_frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness + 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()


Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), no zooming required.
Zooming in to adjust distance, current zoom scale: 1.1
Distance is within acceptable range (80-90 cm), no zooming required.
Distance is within acceptable range (80-90 cm), 

## **Without data**

In [1]:
import cv2
import numpy as np
import time

KNOWN_DISTANCE = 60.0
KNOWN_WIDTH = 16.0
TARGET_MIN_DISTANCE = 80.0
TARGET_MAX_DISTANCE = 90.0
ZOOM_STEP_IN = 0.1
ZOOM_STEP_OUT = 0.1
HEAD_TOP_THRESHOLD_PIXELS = 100

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

cap = cv2.VideoCapture(0)
sample_face_width = 200
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

zoom_scale = 1.0
pulse_alpha = 0.4
pulse_direction = 1
pulse_rate = 0.02

arrow_pulse = 0
arrow_pulse_direction = 1
arrow_pulse_max = 40
arrow_pulse_rate = 2

blink_start_time = time.time()
blink_duration = 1.0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    height, width = frame.shape[:2]
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)
    x_center = width // 2
    y_center = height // 2

    cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                          x_center - new_width // 2:x_center + new_width // 2]
    resized_frame = cv2.resize(cropped_frame, (width, height))
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) > 0:
        largest_face = max(faces, key=lambda face: face[2])
        (x, y, w, h) = largest_face

        distance = estimate_distance(focal_length, KNOWN_WIDTH, w)

        if distance > TARGET_MAX_DISTANCE and zoom_scale < 2.0:
            zoom_scale += ZOOM_STEP_IN
        elif distance < TARGET_MIN_DISTANCE and zoom_scale > 1.0:
            zoom_scale -= ZOOM_STEP_OUT

        head_top_distance = y
        effective_threshold = HEAD_TOP_THRESHOLD_PIXELS * zoom_scale

        if head_top_distance > effective_threshold:
            pulse_alpha += pulse_direction * pulse_rate
            if pulse_alpha >= 0.6 or pulse_alpha <= 0.1:
                pulse_direction *= -1

            overlay = resized_frame.copy()
            cv2.rectangle(overlay, (0, 0), (width, int(head_top_distance)), (0, 0, 255), -1)
            resized_frame = cv2.addWeighted(overlay, pulse_alpha, resized_frame, 1 - pulse_alpha, 0)

            arrow_pulse += arrow_pulse_direction * arrow_pulse_rate
            if arrow_pulse >= arrow_pulse_max or arrow_pulse <= 0:
                arrow_pulse_direction *= -1

            arrow_start = (x + w // 2, y + h + 10)
            arrow_end = (x + w // 2, y + h + 10 + arrow_pulse)
            cv2.arrowedLine(resized_frame, arrow_end, arrow_start, (0, 0, 255), 4, tipLength=0.3)

            text = "Reduce upper distance"
            font_scale = 1
            font_thickness = 2
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
            text_x = (width - text_size[0]) // 2
            text_y = (int(head_top_distance) - text_size[1]) // 2 + text_size[1]

            current_time = time.time()
            if int(current_time - blink_start_time) % 2 == 0:
                cv2.putText(resized_frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness + 2, cv2.LINE_AA)

    cv2.imshow('Distance Estimation with Zoom', resized_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# **Using mediapipe for zooming, facial recognition, head position adjustment**

In [1]:
import cv2
import mediapipe as mp
import numpy as np

KNOWN_DISTANCE = 60.0
KNOWN_WIDTH = 16.0
TARGET_MIN_DISTANCE = 100.0
TARGET_MAX_DISTANCE = 110.0
ZOOM_STEP_IN = 0.05  # Reduced zoom step for smoother transitions
ZOOM_STEP_OUT = 0.05
DAMPING_FACTOR = 0.05  # A small factor to smooth the zoom transitions
ALERT_DISTANCE = 85.0  # Distance threshold for "Too close" alert

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def calculate_focal_length(known_distance, known_width, width_in_image):
    return (width_in_image * known_distance) / known_width

def estimate_distance(focal_length, known_width, width_in_image):
    return (known_width * focal_length) / width_in_image

cap = cv2.VideoCapture(0)

sample_face_width = 200
focal_length = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, sample_face_width)

zoom_scale = 1.0
desired_zoom_scale = zoom_scale  # New variable to store desired zoom level
blink_timer = 0
arrow_timer = 0
color_change_timer = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    new_width = int(width / zoom_scale)
    new_height = int(height / zoom_scale)

    x_center = width // 2
    y_center = height // 2

    cropped_frame = frame[y_center - new_height // 2:y_center + new_height // 2,
                          x_center - new_width // 2:x_center + new_width // 2]
    resized_frame = cv2.resize(cropped_frame, (width, height))

    # Flip the frame horizontally
    flipped_frame = cv2.flip(resized_frame, 1)

    rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_cheek = face_landmarks.landmark[234]
            right_cheek = face_landmarks.landmark[454]
            top_of_head = face_landmarks.landmark[10] 

            width_in_image = int((right_cheek.x - left_cheek.x) * width)
            distance = estimate_distance(focal_length, KNOWN_WIDTH, width_in_image)

            # Display the "Too close" message if distance is below the threshold
            if distance < ALERT_DISTANCE:
                cv2.putText(flipped_frame, "Too Close!", (width // 2 - 100, height // 2), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

            # Adjust desired zoom scale based on the distance
            if distance > TARGET_MAX_DISTANCE and desired_zoom_scale < 2.0:
                desired_zoom_scale += ZOOM_STEP_IN
            elif distance < TARGET_MIN_DISTANCE and desired_zoom_scale > 1.0:
                desired_zoom_scale -= ZOOM_STEP_OUT

            # Gradually update the zoom scale to approach the desired zoom scale
            zoom_scale += (desired_zoom_scale - zoom_scale) * DAMPING_FACTOR

            top_of_head_y = int(top_of_head.y * height)
            distance_from_top = top_of_head_y

            if distance_from_top > 120:
                box_top = 0
                box_bottom = distance_from_top

                color_change_timer += 1
                alpha = (np.sin(color_change_timer * 0.1) + 1) / 2
                red_color = (0, 0, 255)
                blue_color = (255, 0, 0)
                blended_color = (
                    int(red_color[0] * (1 - alpha) + blue_color[0] * alpha),
                    int(red_color[1] * (1 - alpha) + blue_color[1] * alpha),
                    int(red_color[2] * (1 - alpha) + blue_color[2] * alpha)
                )

                overlay = flipped_frame.copy()
                cv2.rectangle(overlay, (0, box_top), (width, box_bottom), blended_color, -1)
                cv2.addWeighted(overlay, 0.3, flipped_frame, 1 - 0.3, 0, flipped_frame)

                line_width_px = int(5 * width / KNOWN_WIDTH)
                cv2.line(flipped_frame, (width // 2 - line_width_px // 2, 40),
                         (width // 2 + line_width_px // 2, 40), (255, 255, 255), 5)

                cv2.putText(flipped_frame, "Adjust Head", (width // 2 - 100, 25), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)

                arrow_size = 80 + 15 * np.sin(arrow_timer * 0.2)
                arrow_timer += 1

                arrow_pts = np.array([
                    [width // 2, top_of_head_y - int(arrow_size)],
                    [width // 2 - 10, top_of_head_y - int(arrow_size / 2)],
                    [width // 2 + 10, top_of_head_y - int(arrow_size / 2)],
                    [width // 2, top_of_head_y],
                ], np.int32)

                cv2.fillPoly(flipped_frame, [arrow_pts], color=(255, 0, 0))

    cv2.imshow('Optimized Face Mesh', flipped_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Disk_D\anaconda\envs\uzair\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
